In [196]:
import numpy as np
import cv2 as cv
import os
import re

In [197]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((6*7, 3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

objpoints = []
imgpoints = []
image_paths = [os.path.join("./samples", path) for path in os.listdir("./samples/")]

for path in image_paths:
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    if ret is True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (7,6), (-1,-1), criteria)
        imgpoints.append(corners2)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx)
np.savez("calibration_matrix.npz", mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

[[533.56891062   0.         341.53147724]
 [  0.         533.69352277 231.41094257]
 [  0.           0.           1.        ]]


In [198]:
img_path = "./samples/left02.jpg"
img = cv.imread(img_path)
h, w = img.shape[:2]
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, None, (w,h), cv.CV_16SC2)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
cv.imwrite(f"./after_undistort/{re.split("[//.]", img_path)[-2]}.png", dst)

True

In [199]:
with np.load("calibration_matrix.npz") as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

img_path = "./with_crosses/02.png"
img = cv.imread(img_path)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, corners = cv.findChessboardCorners(gray, (7,6), None)
 
if ret is True:
    corners2 = cv.cornerSubPix(gray, corners, (7,6), (-1,-1), criteria)
    img_pnts = corners2.reshape(-1, 2)
    ret, rvec, tvec, _ = cv.solvePnPRansac(objp, img_pnts, mtx, dist)

In [200]:
def templateMatching(img_path, template_path, on_black=False):
    img_rgb = cv.imread(img_path)
    assert img_rgb is not None, "file could not be read, check with os.path.exists()"
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
    assert template is not None, "file could not be read, check with os.path.exists()"
    if on_black:
        template[template > 160] = 0
    w, h = template.shape[::-1]
 
    res = cv.matchTemplate(img_gray, template, cv.TM_CCOEFF_NORMED)
    threshold = 0.6
    pos = np.where(res >= threshold)

    # for pt in zip(*pos[::-1]):
    #     cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
    # cv.imwrite('found_cross_template.png', img_rgb)
    
    return [(pt[0] + w//2, pt[1] + h//2) for pt in zip(*pos[::-1])]

In [201]:
def find_max_min_coords(pt1, pt2, pt3, pt4):
    min_x = min(pt1[0], pt2[0], pt3[0], pt4[0])
    max_x = max(pt1[0], pt2[0], pt3[0], pt4[0])
    min_y = min(pt1[1], pt2[1], pt3[1], pt4[1])
    max_y = max(pt1[1], pt2[1], pt3[1], pt4[1])
    return min_x, max_x, min_y, max_y

def find_cross_box(cross_loc, box_locs):
    for i in range(5):
        for j in range(6):
            min_x, max_x, min_y, max_y = find_max_min_coords(box_locs[j + 7*i], 
                                                             box_locs[j + 1 + 7*i], 
                                                             box_locs[j + 7*(i + 1)], 
                                                             box_locs[j + 1 + 7*(i + 1)])
            if min_x <= cross_loc[0] <= max_x and min_y <= cross_loc[1] <= max_y:
                return (j + 0.5, i + 0.5)
    return None

In [202]:
def flatten(xss):
    return [x for xs in xss for x in xs]

template_path = "cross_template_white.png"
cross_locs = []
cross_locs.append(templateMatching(img_path, template_path, True))
cross_locs.append(templateMatching(img_path, template_path))
cross_locs = flatten(cross_locs)

img = cv.imread(img_path)
cross_boxes = []
for pt in cross_locs:
    cross_box = find_cross_box(pt, img_pnts)
    if cross_box is not None:
        cross_boxes.append(cross_box)
cross_boxes = set(cross_boxes)

In [203]:
import math

def get_polygon(sides, radius=1, center=None):
    one_segment = math.pi * 2 / sides
    points = [
        (math.sin(one_segment * i) * radius,
         math.cos(one_segment * i) * radius,
         0)
        for i in range(sides)]
    
    if center:
        points = [[sum(pair) for pair in zip(point, center)]
                  for point in points]
    return points

In [204]:
img = cv.imread(img_path)

for box in cross_boxes:
    center_point = (box[0], box[1], 0)
    points_circle = np.float32(get_polygon(20, 0.4, center_point))
    imgpts, _ = cv.projectPoints(points_circle, rvec, tvec, mtx, dist)
    cv.fillPoly(img, pts=[np.int32(imgpts).reshape(-1,2)], color=(0, 0, 255))

    point_cone = (box[0], box[1], -2)
    for i in range(len(points_circle) - 1):
        triangle = np.float32([points_circle[i], points_circle[i + 1], point_cone])
        imgpts, _ = cv.projectPoints(triangle, rvec, tvec, mtx, dist)
        cv.fillPoly(img, pts=[np.int32(imgpts).reshape(-1,2)], color=(0, 0, 255))

cv.imwrite(f"./res/{re.split("[//.]", img_path)[-2]}.png", img)

True